In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import Dataset
from utils.utils import prepare
from data.configs.demo import config
from collections import defaultdict

from transformers import BertTokenizer,BertModel
from utils.MIND import MIND

logging.getLogger(__name__)

<Logger __main__ (INFO)>

In [2]:
a = torch.load('/data/workspace/Peitian/Code/Document-Reduction/Code/data/model_params/esm__cnn__rnn-u__bm25-reducer__onepass-bert-without-order/demo_step589_[k=3].model')

In [11]:
c = a['optimizer']

In [13]:
c.state_dict()